In [1]:
import sys
sys.path.append("../../hw/base")

from verification_utils import CHCs, mk_int_array

In [2]:
from z3 import *
import z3
z3.set_param(proof=True)  # this is needed so that we can call `.proof()` later

## First example
A loop-free program

In [3]:
stack = Array('stack', IntSort(), IntSort())
sp = Int('sp')
state_vars = [stack, sp]

a, b = input_vars = Ints('a b')

In [4]:
U = {i: Function(f"U{i}", *(v.sort() for v in [*input_vars, *state_vars]), BoolSort())
     for i in range(6)}

In [5]:
sigma = [*input_vars, *state_vars]

"""
{ stack = [a, b] }
PUSH 13
POP 2; ALU MUL
POP 2; ALU ADD
POP 1
{ ret = a + b * 13 }
"""
chcs = CHCs([
    #Implies(And(stack[0] == a, stack[1] == b, sp == 2), U[0](sigma)),
    Implies(And(stack == mk_int_array([a,b]), sp == 2), U[0](sigma)),
    Implies(U[0](sigma), U[1](a, b, Store(stack, sp, 13), sp + 1)),
    Implies(U[1](sigma), U[2](a, b, Store(stack, sp - 2, stack[sp - 1] * stack[sp - 2]), sp - 1)),
    Implies(U[2](sigma), U[3](a, b, Store(stack, sp - 2, stack[sp - 1] + stack[sp - 2]), sp - 1)),
    Implies(U[3](sigma), U[4](a, b, stack, sp - 1)),
    Implies(U[4](sigma), stack[sp] == a + b * 13),
])
chcs

"stack = Store(Store(K(Int, 0), 0, a), 1, b) ∧ sp = 2 ⇒ U0(a, b, stack, sp)"
"U0(a, b, stack, sp) ⇒ U1(a, b, Store(stack, sp, 13), sp + 1)"
"U1(a, b, stack, sp) ⇒ U2(a, b, Store(stack, sp - 2, stack[sp - 1]·stack[sp - 2]), sp - 1)"
"U2(a, b, stack, sp) ⇒ U3(a, b, Store(stack, sp - 2, stack[sp - 1] + stack[sp - 2]), sp - 1)"
"U3(a, b, stack, sp) ⇒ U4(a, b, stack, sp - 1)"
"U4(a, b, stack, sp) ⇒ stack[sp] = a + b·13"


In [6]:
s = chcs.create_solver()
#s.set('xform.inline_eager', False)
#s.set('xform.inline_linear', False)
%time s.check()

CPU times: user 3.18 ms, sys: 507 µs, total: 3.69 ms
Wall time: 3.81 ms


sat

In [7]:
s.model()

[U3 = [else ->
       Or(And(Var(2) ==
              Store(Store(Store(K(Int, 0), 2, 13),
                          1,
                          13*Var(1)),
                    0,
                    13*Var(1) + Var(0)),
              Var(3) == 1),
          And(Var(3) == 1,
              Var(2) ==
              Store(Store(Store(K(Int, 0), 2, 13),
                          1,
                          13*Var(1)),
                    0,
                    13*Var(1) + Var(0))))],
 U2 = [else ->
       Or(And(Var(2) ==
              Store(Store(Store(K(Int, 0), 0, Var(0)),
                          2,
                          13),
                    1,
                    13*Var(1)),
              Var(3) == 2),
          And(Var(3) == 2,
              Var(2) ==
              Store(Store(Store(K(Int, 0), 0, Var(0)),
                          2,
                          13),
                    1,
                    13*Var(1))))],
 U1 = [else ->
       Or(And(Var(2) ==
              Store(Store(Store(K(Int, 0), 0, Var(0)),
                          1,
                          Var(1)),
                    2,
                    13),
              Var(3) == 3),
          And(Var(3) == 3,
              Var(2) ==
              Store(Store(Store(K(Int, 0), 0, Var(0)),
                          1,
                          Var(1)),
                    2,
                    13)))],
 U0 = [else ->
       And(Var(2) ==
           Store(Store(K(Int, 0), 0, Var(0)), 1, Var(1)),
           Var(3) == 2)],
 U4 = [else ->
       Or(And(Var(2) ==
              Store(Store(Store(K(Int, 0), 2, 13),
                          1,
                          13*Var(1)),
                    0,
                    13*Var(1) + Var(0)),
              Var(3) == 0),
          And(Var(3) == 0,
              Var(2) ==
              Store(Store(Store(K(Int, 0), 2, 13),
                          1,
                          13*Var(1)),
                    0,
                    13*Var(1) + Var(0))))]]

## Second Example: A Loop That Computes Addition

The very basic: two input variables, two local variables.

In [8]:
x, y = Ints("x y")

Inv = Function("inv", a.sort(), b.sort(), x.sort(), y.sort(), BoolSort())

chcs = CHCs([
    Implies(And(b > 0), Inv(a, b, a, 0)),
    Implies(And(Inv(a, b, x, y), y < b), Inv(a, b, x + 1, y + 1)),
    Implies(And(Inv(a, b, x, y), y >= b, Not(x == a + b)), False)
])
chcs

"b > 0 ⇒ inv(a, b, a, 0)"
"inv(a, b, x, y) ∧ y < b ⇒ inv(a, b, x + 1, y + 1)"
"inv(a, b, x, y) ∧ y ≥ b ∧ ¬(x = a + b) ⇒ False"


In [9]:
s = chcs.create_solver()
s.check()

sat

In [10]:
# this can be useful if you want to write the Horn clauses to a file
with open('simple.smt2', 'w') as f:
    f.write(f'(set-logic HORN)\n{s.to_smt2()}')

### Stack version: attempt #1

In [11]:
x, y = Ints("x y")
Inv = Function("inv", a.sort(), b.sort(), stack.sort(), sp.sort(), BoolSort())

chcs = CHCs([
    Implies(And(stack == mk_int_array([a,b,a,0]), sp == 4, b > 0), Inv(sigma)),
    Implies(And(Inv(a, b, mk_int_array([a, b, x, y]), sp), y < b),
            Inv(a, b, mk_int_array([a, b, x + 1, y + 1]), sp)),
    Implies(And(Inv(a, b, mk_int_array([a, b, x, y]), sp), y >= b), x == a + b)
])
    
chcs

"stack = Store(Store(Store(Store(K(Int, 0), 0, a), 1, b), 2, a), 3, 0) ∧ sp = 4 ∧ b > 0 ⇒ inv(a, b, stack, sp)"
"inv(a, b, Store(Store(Store(Store(K(Int, 0), 0, a), 1, b), 2, x), 3, y), sp) ∧ y < b ⇒ inv(a, b, Store(Store(Store(Store(K(Int, 0), 0, a), 1, b), 2, x + 1), 3, y + 1), sp)"
"inv(a, b, Store(Store(Store(Store(K(Int, 0), 0, a), 1, b), 2, x), 3, y), sp) ∧ y ≥ b ⇒ x = a + b"


In [12]:
s = chcs.create_solver()
#s.set('xform.inline_eager', False)
#s.set('xform.inline_linear', False)
s.set(timeout=10000)
%time s.check()

CPU times: user 9.93 s, sys: 25.6 ms, total: 9.96 s
Wall time: 10 s


unknown

This did not work. The explicit array creation seems to be too much for Spacer to solve. While it is hard to find the root cause of the divergence, we might wish to explore alternative encodings of the problem.

### Stack version: attempt #2

Here we use two input variables `a`, `b` and a stack for the temp vars. `x` is stored at `stack[0]` and `y` is stored at `stack[1]`.

In [13]:
Inv = Function('inv', a.sort(), b.sort(), stack.sort(), BoolSort())

chcs = CHCs([
    Implies(And(b > 0, stack[0] == a, stack[1] == 0), Inv(a, b, stack)),
    Implies(And(Inv(a, b, stack), stack[0] == x, stack[1] == y, y < b), Inv(a, b, Store(Store(stack, 0, x + 1), 1, y + 1))),
    Implies(And(Inv(a, b, stack), stack[0] == x, stack[1] == y, y >= b), x == a + b)
])
chcs

"b > 0 ∧ stack[0] = a ∧ stack[1] = 0 ⇒ inv(a, b, stack)"
"inv(a, b, stack) ∧ stack[0] = x ∧ stack[1] = y ∧ y < b ⇒ inv(a, b, Store(Store(stack, 0, x + 1), 1, y + 1))"
"inv(a, b, stack) ∧ stack[0] = x ∧ stack[1] = y ∧ y ≥ b ⇒ x = a + b"


In [14]:
s = chcs.create_solver()
#s.set('xform.inline_eager', False)
#s.set('xform.inline_linear', False)
(res := s.check())

sat

In [15]:
from verification_utils import HyperResolutionProof
HyperResolutionProof(s.proof()).to_roadmap()    if res == unsat else s.model()

[inv = [else ->
        And(Not(Var(2)[0] + -1*Var(0) + -1*Var(2)[1] <= -1),
            Not(Var(2)[0] + -1*Var(0) + -1*Var(2)[1] >= 1),
            Not(Var(1) + -1*Var(2)[0] + Var(0) <= -1))]]

Cool. Now let's make it gradually more realistic by expressing our proper stack semantics.

### Stack version: using `sp`
Very similar to before, but variable offsets are relative to `sp`, like in StaM.

In [16]:
Inv = Function('inv', a.sort(), b.sort(), stack.sort(), sp.sort(), BoolSort())

chcs = CHCs([
    Implies(And(b > 0, stack[sp - 2] == a, stack[sp - 1] == 0), Inv(a, b, stack, sp)),
    Implies(And(Inv(a, b, stack, sp), stack[sp - 2] == x, stack[sp - 1] == y, y < b),
            Inv(a, b, Store(Store(stack, sp - 2, x + 1), sp - 1, y + 1), sp)),
    Implies(And(Inv(a, b, stack, sp), stack[sp - 2] == x, stack[sp - 1] == y, y >= b), x == a + b)
])
chcs

"b > 0 ∧ stack[sp - 2] = a ∧ stack[sp - 1] = 0 ⇒ inv(a, b, stack, sp)"
"inv(a, b, stack, sp) ∧ stack[sp - 2] = x ∧ stack[sp - 1] = y ∧ y < b ⇒ inv(a, b, Store(Store(stack, sp - 2, x + 1), sp - 1, y + 1), sp)"
"inv(a, b, stack, sp) ∧ stack[sp - 2] = x ∧ stack[sp - 1] = y ∧ y ≥ b ⇒ x = a + b"


In [17]:
s = chcs.create_solver()
(res := s.check())

sat

In [18]:
HyperResolutionProof(s.proof()).to_roadmap()    if res == unsat else None

## Stack version: all in the stack
Also similar, but the input arguments `a` and `b` are now also stored on the stack.

In [19]:
Inv = Function('inv', stack.sort(), sp.sort(), BoolSort())

p = And(stack[sp - 4] == a, stack[sp - 3] == b)

chcs = CHCs([
    Implies(And(p, b > 0, stack[sp - 2] == a, stack[sp - 1] == 0), Inv(stack, sp)),
    Implies(And(p, Inv(stack, sp), stack[sp - 2] == x, stack[sp - 1] == y, y < b),
            Inv(Store(Store(stack, sp - 2, x + 1), sp - 1, y + 1), sp)),
    Implies(And(p, Inv(stack, sp), stack[sp - 2] == x, stack[sp - 1] == y, y >= b), x == a + b)
])
chcs

"stack[sp - 4] = a ∧ stack[sp - 3] = b ∧ b > 0 ∧ stack[sp - 2] = a ∧ stack[sp - 1] = 0 ⇒ inv(stack, sp)"
"stack[sp - 4] = a ∧ stack[sp - 3] = b ∧ inv(stack, sp) ∧ stack[sp - 2] = x ∧ stack[sp - 1] = y ∧ y < b ⇒ inv(Store(Store(stack, sp - 2, x + 1), sp - 1, y + 1), sp)"
"stack[sp - 4] = a ∧ stack[sp - 3] = b ∧ inv(stack, sp) ∧ stack[sp - 2] = x ∧ stack[sp - 1] = y ∧ y ≥ b ⇒ x = a + b"


In [20]:
s = chcs.create_solver()
(res := s.check())

sat

## Stack version: money time; doing it in assembly

Now let's write the function's implementation in StASM, and observe the CHCs that encode its behavior. Here we do the translation manually. In your project, you will generate those clauses automatically from the StASM code.

In [21]:
"""
{ stack = [a, b] ∧ b > 0 }
  DUP 1
  PUSH 0
loop:
  DUP; DUP 2; POP 2; ALU LT; POP 1; JZ exit
  DUP 1; PUSH 1; POP 2; ALU ADD
  DUP 1; PUSH 1; POP 2; ALU ADD
  YANK 2,2
  JMP loop
exit:
  POP 1
{ stack = [a, b, a + b] }
"""

sigma = [*state_vars]

U = {i: Function(f"U{i}", *(v.sort() for v in sigma), BoolSort())
     for i in range(9)}

chcs = CHCs([
    # assume { stack = [a, b] ∧ b > 0 }
    Implies(stack[sp - 1] > 0, U[0](sigma)),
    # DUP 1; PUSH 0
    Implies(U[0](sigma), U[1](Store(Store(stack, sp, stack[sp - 2]), sp + 1, 0), sp + 2)),
    # DUP; DUP 2
    Implies(U[1](sigma), U[2](Store(Store(stack, sp, stack[sp - 1]), sp + 1, stack[sp - 3]), sp + 2)),
    # POP 2; ALU LT; POP 1; JZ exit
    Implies(And(U[2](sigma), stack[sp - 2] < stack[sp - 1]), U[3](stack, sp - 2)),
    Implies(And(U[2](sigma), stack[sp - 2] >= stack[sp - 1]), U[7](stack, sp - 2)),
    # DUP 1; PUSH 1; POP 2; ALU ADD
    Implies(U[3](sigma), U[4](Store(stack, sp, stack[sp - 2] + 1), sp + 1)),
    # DUP 1; PUSH 1; POP 2; ALU ADD
    Implies(U[4](sigma), U[5](Store(stack, sp, stack[sp - 2] + 1), sp + 1)),
    # YANK 2,2
    Implies(U[5](sigma), U[6](Store(Store(stack, sp - 4, stack[sp - 2]), sp - 3, stack[sp - 1]), sp - 2)),
    # JMP loop
    Implies(U[6](sigma), U[1](sigma)),
    # exit:
    # POP 1
    Implies(U[7](sigma), U[8](stack, sp - 1)),
    # assert { stack = [a, b, a + b] }
    Implies(U[8](sigma), stack[sp - 1] == stack[sp - 3] + stack[sp - 2])
])
chcs

"stack[sp - 1] > 0 ⇒ U0(stack, sp)"
"U0(stack, sp) ⇒ U1(Store(Store(stack, sp, stack[sp - 2]), sp + 1, 0), sp + 2)"
"U1(stack, sp) ⇒ U2(Store(Store(stack, sp, stack[sp - 1]), sp + 1, stack[sp - 3]), sp + 2)"
"U2(stack, sp) ∧ stack[sp - 2] < stack[sp - 1] ⇒ U3(stack, sp - 2)"
"U2(stack, sp) ∧ stack[sp - 2] ≥ stack[sp - 1] ⇒ U7(stack, sp - 2)"
"U3(stack, sp) ⇒ U4(Store(stack, sp, stack[sp - 2] + 1), sp + 1)"
"U4(stack, sp) ⇒ U5(Store(stack, sp, stack[sp - 2] + 1), sp + 1)"
"U5(stack, sp) ⇒ U6(Store(Store(stack, sp - 4, stack[sp - 2]), sp - 3, stack[sp - 1]), sp - 2)"
"U6(stack, sp) ⇒ U1(stack, sp)"
"U7(stack, sp) ⇒ U8(stack, sp - 1)"
"U8(stack, sp) ⇒ stack[sp - 1] = stack[sp - 3] + stack[sp - 2]"


In [22]:
s = chcs.create_solver()
s.set('fp.spacer.global', True)    # *THIS IS REQUIRED*
#s.set('xform.inline_eager', False)
#s.set('xform.inline_linear', False)
%time (res := s.check())

CPU times: user 58.6 ms, sys: 5.54 ms, total: 64.2 ms
Wall time: 64.2 ms


sat

In [23]:
HyperResolutionProof(s.proof()).to_roadmap()    if res == unsat else s.model()

[U1 = [else ->
       Or(Exists([k!1, k!0],
                 And(Not(k!1[-1 + k!0] <= 0),
                     Var(2) ==
                     Store(Store(k!1, k!0, k!1[-2 + k!0]),
                           1 + k!0,
                           0),
                     Var(3) == 2 + k!0)),
          Exists([k!3, k!2],
                 And(Not(k!3[-3 + k!2] +
                         -1*k!3[-4 + k!2] +
                         k!3[-6 + k!2] >=
                         1),
                     Not(k!3[-3 + k!2] +
                         -1*k!3[-4 + k!2] +
                         k!3[-6 + k!2] <=
                         -1),
                     Or(Not(k!3[-2 + k!2] + -1*k!3[-1 + k!2] <=
                            -1),
                        Not(k!3[-5 + k!2] +
                            -1*k!3[-4 + k!2] +
                            k!3[-6 + k!2] <=
                            0)),
                     Not(k!3[-5 + k!2] +
                         -1*k!3[-4 + k!2] +
                         k!3[-6 + k!2] <=
                         -1),
                     Not(k!3[-5 + k!2] +
                         -1*k!3[-4 + k!2] +
                         k!3[-2 + k!2] +
                         -1*k!3[-1 + k!2] +
                         k!3[-6 + k!2] >=
                         1),
                     Var(2) ==
                     Store(Store(Store(Store(k!3,
                                        -2 + k!2,
                                        1 + k!3[-4 + k!2]),
                                       -1 + k!2,
                                       1 +
                                       Store(k!3,
                                        -2 + k!2,
                                        1 + k!3[-4 + k!2])[-3 +
                                        k!2]),
                                 -4 + k!2,
                                 Store(Store(k!3,
                                        -2 + k!2,
                                        1 + k!3[-4 + k!2]),
                                       -1 + k!2,
                                       1 +
                                       Store(k!3,
                                        -2 + k!2,
                                        1 + k!3[-4 + k!2])[-3 +
                                        k!2])[-2 + k!2]),
                           -3 + k!2,
                           1 +
                           Store(k!3,
                                 -2 + k!2,
                                 1 + k!3[-4 + k!2])[-3 + k!2]),
                     Not(k!3[-1 + k!2] <= k!3[-2 + k!2]),
                     Var(3) == -2 + k!2)),
          Exists(k!2,
                 And(Not(k!2[-3 + Var(2)] <= 0),
                     Var(1) ==
                     Store(Store(k!2,
                                 -2 + Var(2),
                                 k!2[-4 + Var(2)]),
                           -1 + Var(2),
                           0))))],
 U3 = [else ->
       Exists(k!0,
              And(Not(Var(1)[-3 + k!0] +
                      -1*Var(1)[-4 + k!0] +
                      Var(1)[-6 + k!0] >=
                      1),
                  Not(Var(1)[-3 + k!0] +
                      -1*Var(1)[-4 + k!0] +
                      Var(1)[-6 + k!0] <=
                      -1),
                  Or(Not(Var(1)[-2 + k!0] +
                         -1*Var(1)[-1 + k!0] <=
                         -1),
                     Not(Var(1)[-5 + k!0] +
                         -1*Var(1)[-4 + k!0] +
                         Var(1)[-6 + k!0] <=
                         0)),
                  Not(Var(1)[-5 + k!0] +
                      -1*Var(1)[-4 + k!0] +
                      Var(1)[-6 + k!0] <=
                      -1),
                  Not(Var(1)[-5 + k!0] +
                      -1*Var(1)[-4 + k!0] +
                      Var(1)[-2 + k!0] +
                      -1*Var(1)[-1 + k!0] +
                      Var(1)[-6 + k!0] >=
                      1),
                  Not(Var(1)[-1 + k!0] <= Var(

### Exercise
Modify some nit in the CHCs for the program above, simulating a buggy program. The result should be `unsat`. Generate the trace with the `HyperResolutionProof` shown above (which is like we did in hardware model checking).
Try to uncomment the lines setting the flags `xform.inline_eager` and `xform.inline_linear`, disabling some of Spacer's preprocessing optimizations, in order to get a fuller trace.
If you are getting large values for `sp` (_e.g._ 21353), add a constraint somewhere that delimits its range for, say, 0..10.